In [1]:
from PIL import Image
import tqdm
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
ORIGINAL_IMAGE_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/elias_phone_all'
REDACTED_FOLDER = '/home/datasets/ecg-digitization/digital_redaction/elias_all_redacted_final'
REDACTION_HIGHLIGHTED_SAVE_FOLDER = '/home/datasets/ecg-digitization/redacted/elias_highlighted_digital_redactions'
REDACTION_SAVE_FOLDER = '/home/datasets/ecg-digitization/redacted/elias_digital_redactions'

REDACTION_COLOR = np.array([0, 255, 0])
REDACTION_NORM_TRESHOLD = 50
OUTPUT_REDACTION_COLOR = np.array([0, 0, 0])

DEBUG = False


os.makedirs(REDACTION_HIGHLIGHTED_SAVE_FOLDER, exist_ok=True)
os.makedirs(REDACTION_SAVE_FOLDER, exist_ok=True)


image_names = tqdm.tqdm(os.listdir(REDACTED_FOLDER), desc='Redacting images', unit='image')

for image_name in image_names:
    redacted_path = os.path.join(REDACTED_FOLDER, image_name)
    original_path = os.path.join(ORIGINAL_IMAGE_FOLDER, image_name)

    img = Image.open(original_path)
    img_np = np.array(img, dtype=np.int64)

    redacted_img = Image.open(redacted_path)
    redacted_img_np = np.array(redacted_img, dtype=np.int64)

    redaction_distance = np.linalg.norm(redacted_img_np - REDACTION_COLOR.reshape(1, 1, 3), axis=-1)
    redaction_mask = redaction_distance < REDACTION_NORM_TRESHOLD

    if DEBUG:
        plt.figure()
        plt.hist(redaction_distance.flatten(), bins=256, color='green', alpha=0.7)
        plt.title('Histogram of Redacted Image Pixel Values')
        plt.xlabel('Pixel Value')
        plt.ylabel('Frequency')
        plt.grid()

    img_np[redaction_mask] = REDACTION_COLOR
    Image.fromarray(img_np.astype(np.uint8)).save(os.path.join(REDACTION_HIGHLIGHTED_SAVE_FOLDER, image_name))

    img_np[redaction_mask] = OUTPUT_REDACTION_COLOR
    Image.fromarray(img_np.astype(np.uint8)).save(os.path.join(REDACTION_SAVE_FOLDER, image_name))


Redacting images: 100%|██████████| 558/558 [06:33<00:00,  1.42image/s]
